# 学習したモデルによる予測
---
### 利用バージョン
python 2 (python 3にしたかったけど、色々環境設定しないといけなさそうなので・・)

## 概要
ここでは、
* リサイズする解像度のサイズ、予測させたい画像ファイルのパス指定
* 予測結果の確認

といった事をします。

## 変数定義
* リサイズする解像度をIMG_SIZEにて指定。これは学習時に使用したサイズと合わせる必要があります。
* 予測したい画像をFILE_NAMEで指定します。

In [1]:
IMG_SIZE = 250
FILE_NAME = u'IMG_4071.JPG'

## 予測スタート

In [44]:
from PIL import Image
from glob import glob
import os
import numpy as np
import pandas as pd
import chainer
from chainer import datasets
from chainer import iterators
import chainer.links as L
import chainer.functions as F
from chainer import optimizers
from chainer import training
from chainer.training import extensions
from chainer import serializers
import sys

def resizePict(filePathName):
    pflag = False
    
    # 画像ファイルとして開く&リサイズできるか試す
    pictFile = Image.open(filePathName)
    reSizePictFile = pictFile.resize((IMG_SIZE, IMG_SIZE))
 
    # RGB形式ではない場合、RGB形式に変換する。
    if reSizePictFile.mode != 'RGB':
        reSizePictFile = reSizePictFile.convert('RGB')

    # リサイズしたファイルを出力する。
    #outFilePathName = os.path.join(filePath, u'resize_{0}'.format(FILE_NAME))
    #reSizePictFile.save(outFilePathName)
    
    return reSizePictFile

def transform(data):
    return data / 255.

# AIのモデル定義
class MLP(chainer.Chain):

    def __init__(self, n_mid_units=100, n_out=2):
        # パラメータを持つ層の登録
        super(MLP, self).__init__(
            l1=L.Linear(None, n_mid_units),
            l2=L.Linear(None, n_mid_units),
            l3=L.Linear(None, n_out),
        )

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)
    
def main():
    filePathName = os.path.join(u'dataset', u'04_samplePict', FILE_NAME)
    
    pictFile = resizePict(filePathName)
    
    #dataset = datasets.LabeledImageDataset(outFilePathName)
    # from IPython.display import Image
    # Image(filename=filePathName)
    
    # PILは(color, x, y)、Chainerは(x, y, color)の並び順にする必要があるため、transposeで変換
    # 画素値を255で割り、0〜1の値にする
    x = np.asarray(pictFile).transpose(2, 0, 1).astype(np.float32)/255.
    
    # ミニバッチの形に変換
    x = x[None, ...]
    
    # モデルの定義
    model = MLP()
    model = L.Classifier(model)
    #model.to_gpu(gpu_id)
    serializers.load_npz(u'result/sakamotsu.model', model)
    t = model.predictor(x)
    y = F.softmax(t)
    ans = y.data.argmax(axis=1)[0]
    print (u'予測値:{}'.format(ans))
    print (u'好き(1)の可能性　:{:.2%}'.format(y.data[0][1]))
    print (u'嫌い(0)の可能性　:{:.2%}'.format(y.data[0][0]))
    
if __name__ == '__main__':
    main()

予測値:1
好き(1)の可能性　:58.41%
嫌い(0)の可能性　:41.59%
